In [ ]:
# ------------------------------------------------------------------------------
#     Copyright 2022 Google LLC. All Rights Reserved.
#
#     Licensed under the Apache License, Version 2.0 (the "License");
#     you may not use this file except in compliance with the License.
#     You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0
#
#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.
# ------------------------------------------------------------------------------

#@markdown #  Train your own DDSP-VST Model 
#@markdown 🎻🎺🎸🎵 [g.co/magenta/train-ddsp-vst](g.co/magenta/train-ddsp-vst)

#@markdown <br/> 

#@markdown ## Instructions


#@markdown * Create a folder in Google Drive with your training audio (`.wav` or `.mp3`)

Name = 'My Instrument' #@param {type:"string"}
Name = Name.replace(' ', '_')


#@markdown * Press the ▶️ button in the upper left!

#@markdown * Login to your Google account when asked

#@markdown *  Select your folder with the file chooser below when asked

#@markdown *  Wait (with this window open) for training to finish and download the model

#@markdown *  If something breaks, resume training by refreshing this page, press ▶️, and choose the same folder



#@markdown <br/>

#@markdown <br/>

#@markdown ## Data
#@markdown Custom models can train on as little as 10 minutes of audio (`.wav` or `.mp3`). You can get the best results from "monophonic" (only one note at a time) audio from a single recording session (same mic, same reverb). All of your data is private, used locally, and erased as soon as your colab session ends.


#@markdown ## Training
#@markdown Training typically takes ~2-3 hours with free Colab, and less than an hour with ColabPro+. Free colab can sometimes disconnects before models finish training, but there are some unofficial [ways around this](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting). If you do get disconnected, don't worry, just press play again and choose the same folder. The training will resume where it left off.



#@markdown ## Export

#@markdown After training, it should automatically export and download your model as `{my_name}.tflite` that you can use by dropping in the VST custom models folder, which you can access by pushing the 'Models' button within the plugin itself. If it doesn't automatically download, you can find the file in the `ddsp-training-{date-time}/export` folder, either on this page (click the 📁 icon on the left), or in the folder you selected from your drive.

##@markdown We recommend using Google Drive to load data faster and save your model during training. Just create a folder on your drive with your audio files in it, and select the folder. If you don't use drive, you can still upload audio through the browser (slower) and download the final trained model.



#@markdown <br/> <br/>
#@markdown ## Advanced Options

##@markdown <a href="https://colab.research.google.com/github/magenta/ddsp/blob/main/ddsp/colab/demos/Train_VST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#@markdown <br/>
#@markdown  Usually we will produce good results after training between 30k-50k steps, but your results may vary depending on your audio files/instrument. Too few steps will often have the model sound bland/generic, too many steps can often lead to more "sputtering" and big volume fluctuations

Training_Steps = 30000 #@param {type:"integer"}

#@markdown <br/>
#@markdown Ignore previous checkpoints in the folder and start a fresh run from step 0

Ignore_Previous = False #@param {type:"boolean"}


#@markdown <br/>
#@markdown Use Google Drive for training? Otherwise, loads audio from browser, which is much slower
Google_Drive = True #@param {type:"boolean"}



# Sample_Rate = '16kHz'  #@param ['16kHz', '32kHz', '48kHz']
# Sample_Rate = {'16kHz': 16000, '32kHz': 32000, '48kHz': 48000}[Sample_Rate]
# Model_Gin_File = 'models/vst/vst.gin'




# ------------------------------------------------------------------------------
# Install
# ------------------------------------------------------------------------------
print('Installing DDSP...')
print('This should take about 2 minutes...')
!pip install -U ddsp[data_preparation]==3.4.0 &> /dev/null
!pip install ipyfilechooser &> /dev/null


# ------------------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------------------
print('Importing Libraries...')
print()
import datetime
import glob
import os
import shutil

from ddsp import spectral_ops
from ddsp.colab import colab_utils
import ddsp.training
import gin
from google.colab import drive
from ipyfilechooser import FileChooser
import pydub
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

from ddsp.training.data_preparation.prepare_tfrecord_lib import _load_audio_as_array as load_audio


# ------------------------------------------------------------------------------
# Functions
# ------------------------------------------------------------------------------
def directory_has_files(target_dir):
  n_files = len(glob.glob(os.path.join(target_dir, '*')))
  return n_files > 0


def get_audio_files(drive_dir, audio_dir):
  if drive_dir:
    mp3_files = glob.glob(os.path.join(drive_dir, '*.mp3'))
    wav_files = glob.glob(os.path.join(drive_dir, '*.wav'))
    audio_paths = mp3_files + wav_files
    if len(audio_paths) < 1:
      raise FileNotFoundError("Sorry, it seems that there aren't any MP3 or "
                              f"WAV files in your folder ({drive_dir}). Try "
                              "running again and choose a different folder.")
  else:
    audio_paths, _ = colab_utils.upload()

  # Copy Audio.
  for src in audio_paths:
    target = os.path.join(audio_dir, 
                          os.path.basename(src).replace(' ', '_'))
    print('Copying {} to {}'.format(src, target))
    shutil.copy(src, target)
    # !cp $src $target


def prepare_dataset(audio_dir, 
                    data_dir,
                    sample_rate=16000, 
                    frame_rate=50, 
                    example_secs=4.0, 
                    hop_secs=1.0, 
                    viterbi=True, 
                    center=True):
  if directory_has_files(data_dir):
    print(f'Dataset already exists in `{data_dir}`')
    return
  else:
    # Otherwise prepare new dataset locally.
    print(f'Preparing new dataset from `{audio_dir}`')

    print()
    print('Creating dataset...')
    print('This usually takes around 2-3 minutes for each minute of audio')
    print('(10 minutes of training audio -> 20-30 minutes)')

    audio_filepattern = os.path.join(audio_dir, '*')
    !ddsp_prepare_tfrecord \
    --input_audio_filepatterns=$audio_filepattern \
    --output_tfrecord_path=$data_dir/train.tfrecord \
    --num_shards=10 \
    --sample_rate=$sample_rate \
    --frame_rate=$frame_rate \
    --example_secs=$example_secs \
    --hop_secs=$hop_secs \
    --viterbi=$viterbi \
    --center=$center \
    --alsologtostderr &> /dev/null


def train(model_dir, data_dir, steps=30000):
  file_pattern = os.path.join(data_dir, 'train.tfrecord*')
  !ddsp_run \
  --mode=train \
  --save_dir="$model_dir" \
  --gin_file=models/vst/vst.gin \
  --gin_file=datasets/tfrecord.gin \
  --gin_param="TFRecordProvider.file_pattern='$file_pattern'" \
  --gin_param="TFRecordProvider.centered=True" \
  --gin_param="TFRecordProvider.frame_rate=50" \
  --gin_param="batch_size=16" \
  --gin_param="train_util.train.num_steps=$steps" \
  --gin_param="train_util.train.steps_per_save=300" \
  --gin_param="trainers.Trainer.checkpoints_to_keep=3"

  # --gin_param="train.data_provider=@ExperimentalDataProvider()" \
  # --gin_param="ExperimentalRecordProvider.data_dir='$data_dir'" \
  # --gin_param="ExperimentalRecordProvider.sample_rate=16000" \
  # --gin_param="ExperimentalRecordProvider.frame_rate=50" \


def launch_tensorboard(save_dir):
  %reload_ext tensorboard
  import tensorboard as tb
  tb.notebook.start('--logdir "{}"'.format(save_dir))


def reset_state(data_dir, audio_dir, model_dir):
  if tf.io.gfile.exists(data_dir):
    !rm -r $data_dir
    !rm -r $audio_dir
  !mkdir -p $data_dir
  !mkdir -p $audio_dir
  !mkdir -p $model_dir


def export_and_download(model_dir, model_name=Name):
  export_path = os.path.join(model_dir, 'export')

  !ddsp_export \
  --model_path=$model_dir \
  --save_dir=$export_path \
  --inference_model=vst_stateless_predict_controls \
  --tflite \
  --notfjs

  # Just copy the tflite model.
  tflite_fp = os.path.join(export_path, 'tflite', 'model.tflite')
  my_model = os.path.join(model_dir, f'{model_name}.tflite')
  !cp $tflite_fp $my_model
  print('Export Complete! Downloading...')
  print(f'You can also find your model at {my_model}')
  colab_utils.download(my_model)

  # Copy the whole directory.
  # my_model = f'{model_name}.zip'
  # !zip -r $my_model $export_path
  # colab_utils.download(my_model)


def get_model_dir(base_dir):
  base_str = 'ddsp-training'
  dirs = tf.io.gfile.glob(os.path.join(base_dir, f'{base_str}-*'))
  if dirs and not Ignore_Previous:
    model_dir = dirs[-1]  # Sorted, so last is most recent.
  else:
    now = datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
    model_dir = os.path.join(base_dir, f'{base_str}-{now}')
  return model_dir




def run_training(drive_dir=''):

  # ------------------------------------------------------------------------------
  # Setup
  # ------------------------------------------------------------------------------
  # Save data locally, but model on drive.
  data_dir = 'data/'
  audio_dir = 'audio/'
  model_dir = get_model_dir(drive_dir)

  reset_state(data_dir, audio_dir, model_dir)

  # ------------------------------------------------------------------------------
  # Dataset
  # ------------------------------------------------------------------------------
  get_audio_files(drive_dir, audio_dir)
  prepare_dataset(audio_dir, data_dir)

  # ------------------------------------------------------------------------------
  # Train
  # ------------------------------------------------------------------------------
  print()
  print('Training...')
  train(model_dir, data_dir, steps=Training_Steps)

  # ------------------------------------------------------------------------------
  # Export
  # ------------------------------------------------------------------------------
  print()
  print('Exporting model...')
  export_and_download(model_dir)


def run(Google_Drive=True):
  """Create and display a FileChooser widget."""

  if Google_Drive:
    print('Mounting Google Drive...')
    drive.mount('gdrive', force_remount=True, timeout_ms=10000)    
    initial_dir = 'gdrive/MyDrive'

    def run_after_select(chooser):
      drive_dir = chooser.selected_path
      run_training(drive_dir=drive_dir)

    fc = FileChooser(initial_dir)
    fc.show_only_dirs = True
    fc.title = '<b>Pick a folder with the audio files for training...</b>'
    fc.register_callback(run_after_select)
    display(fc)


  else:
    print('Skipping Drive Setup...')
    print('Upload Audio Manually...')
    run_training(drive_dir='')


run(Google_Drive)
